# Bioinformatics With Python Cookbook
## Cap. 2 - NGS

Neste capítulo será utilizado um conjunto de dados advindo de WGS e o projeto 1000 Genomes. 


Antes de iniciar com os dados reais, vamos ficar confortáveis em acessar banco de dados genomicos existentes, além de proceder ao processamento básico de sequências.  


## Acessando o GenBank e NCBI

In [1]:
from Bio import Entrez, SeqIO

In [2]:
# E-mail usado para acessar Entrez.
Entrez.email = 'araujowash@gmail.com'

In [4]:
# Lista de bancos de dados disponíveis
handle = Entrez.einfo()
rec = Entrez.read(handle)
print(rec)

DictElement({'DbList': ['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'nucgss', 'nucest', 'structure', 'sparcle', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'clone', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'probe', 'proteinclusters', 'pcassay', 'biosystems', 'pccompound', 'pcsubstance', 'pubmedhealth', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'unigene', 'gencoll', 'gtr']}, attributes={})


Achar **Cholroquine Resistance Transporter - CRT**, gene de _Plasmodium falciparum_ no banco de dados de nucleotídeos.

In [5]:
handle = Entrez.esearch(db = 'nucleotide', term = 'CRT[Gene Name] AND "Plasmodium falciparum"[Organism]')

rec_list = Entrez.read(handle)
if rec_list['RetMax'] < rec_list['Count']:
    handle = Entrez.esearch(db='nucleotide', term='CRT[Gene Name] AND "Plasmodium falciparum"[Organism]', 
                            retmax=rec_list['Count'])
    rec_list = Entrez.read(handle)


1. Inicia-se pesquisando no banco de dados colocando o nome do gene e do organismo.   
Então, lê-se o resultado retornado.     
O número de referências retornado é 20 em uma busca padrão, então, se você quer mais, repita a busca (_query_) usando um limite máximo aumentado.  
Neste nosso caso, nós sobrescreveremos o limite padrão com **retmax**.  

2. O sistema Entrez provê várias formas sofisticadas de recuperar grande número de resultados.   
Embora agora tenhamos os IDs de todos os registros, ainda é necessário recupearar os registros propriamente ditos. 

3. Vamos tentar recuperar todos estes registros (_records_).   
A seguinte _query_ fará o _download_ de todas as sequências de nucleotídeos correspondentes no GenBank.

In [6]:
id_list = rec_list['IdList']
hdl = Entrez.efetch(db = 'nucleotide', id = id_list, rettype = 'gb')

Bem, neste caso, vá em frente e faça. No entanto, tenha cuidado com essa técnica, porque você recuperará uma grande quantidade de registros completos e alguns deles terão seqüências razoavelmente grandes. Você corre o risco de baixar muitos dados (ambos são uma sobrecarga do seu lado e em servidores NCBI).

Existem várias maneiras de contornar isso. Uma maneira é fazer uma consulta mais restritiva e / ou baixar apenas alguns de cada vez e parar quando você encontrar o que é suficiente. A estratégia precisa dependerá do que você está tentando alcançar. Em qualquer caso, vamos recuperar uma lista de registros no formato GenBank (que inclui sequências e muitos metadados interessantes).  

4. **Vamos ler e fazer _parse_ dos resultados:**

In [7]:
recs = list(SeqIO.parse(hdl, 'gb'))

Observe que convertemos um iterador (o resultado de SeqIO.parse) em uma lista. A vantagem é que podemos usar o resultado quantas vezes quisermos (por exemplo, iterar várias vezes), sem repetir a consulta no servidor. Isso economiza tempo, largura de banda e uso do servidor se você planeja iterar várias vezes.

A desvantagem é que ele alocará memória para todos os registros. Isso não funcionará para conjuntos de dados muito grandes; talvez você não queira fazer esse tipo de _Genome Wide_ no próximo capítulo. Voltaremos a este tópico na última parte do livro.

Se você está fazendo computação interativa, provavelmente preferirá ter uma lista (para que você possa analisar e experimentar várias vezes), mas se você estiver desenvolvendo uma biblioteca, um iterador provavelmente será a melhor abordagem.

5. Iremos nos concentrar em um registro simples.   
Isso só funcionará se você utilizar exatamente a mesma _query_:

In [8]:
for rec in recs:
    if rec.name == 'KM288867':
        break
        
# The rec variable now has our record of interest. 
# The rec.description will contain its human-readable description.
print(rec.name)
print(rec.description)

KM288867
Plasmodium falciparum clone PF3D7_0709000 chloroquine resistance transporter (CRT) gene, complete cds


6. Vamos extrair algumas características de sequências (produtos gênicos e posições de exons).

In [9]:
for feature in rec.features:
    if feature.type == 'gene':
        print(feature.qualifiers['gene'])
    elif feature.type == 'exon':
        loc = feature.location
        print('Exon', loc.start, loc.end, loc.strand)
    else:
        print('Não processado:\n%s' % feature)

Não processado:
type: source
location: [0:10000](+)
qualifiers:
    Key: clone, Value: ['PF3D7_0709000']
    Key: db_xref, Value: ['taxon:5833']
    Key: mol_type, Value: ['genomic DNA']
    Key: organism, Value: ['Plasmodium falciparum']

['CRT']
Não processado:
type: mRNA
location: join{[2751:3543](+), [3720:3989](+), [4168:4341](+), [4513:4646](+), [4799:4871](+), [4994:5070](+), [5166:5249](+), [5376:5427](+), [5564:5621](+), [5769:5862](+), [6055:6100](+), [6247:6302](+), [6471:7598](+)}
qualifiers:
    Key: gene, Value: ['CRT']
    Key: product, Value: ['chloroquine resistance transporter']

Não processado:
type: 5'UTR
location: [2751:3452](+)
qualifiers:
    Key: gene, Value: ['CRT']

Não processado:
type: primer_bind
location: [2935:2958](+)
qualifiers:

Não processado:
type: primer_bind
location: [3094:3121](+)
qualifiers:

Não processado:
type: CDS
location: join{[3452:3543](+), [3720:3989](+), [4168:4341](+), [4513:4646](+), [4799:4871](+), [4994:5070](+), [5166:5249](+), [5

Se o tipo de recurso for gene, imprimiremos seu nome, que estará no dicionário de qualificadores.

Também vamos imprimir todos os locais dos exons. Os exons, assim como todos os recursos, têm localizações nessa sequência: um começo, um fim e a fita de onde são lidos. Embora todas as posições inicial e final de nossos exons sejam `ExactPosition`, observe que o Biopython suporta muitos outros tipos de posições.

Um tipo de posição é `BeforePosition`, que especifica que um ponto de localização é antes de uma determinada posição de sequência. Outro tipo de posição é `BetweenPosition`, que fornece o intervalo para um determinado local de início / fim. Existem mais alguns tipos de posição; estas são apenas alguns exemplos. As coordenadas serão especificadas para que você possa recuperar a sequência de um array Python com intervalos facilmente, então geralmente o início será um antes que o valor no registro e o final sejam iguais. A questão dos sistemas de coordenadas será revisitada em futuras receitas.


Para outros tipos de recursos, nós simplesmente os imprimimos. Observe que o Biopython fornecerá uma versão legível do recurso quando você imprimi-lo.

7. Observaremos as anotações nos registros, as quais são principalmente metadados que não estão relacionadas às posições de sequências. 

In [11]:
for name, value in rec.annotations.items():
    print('%s = %s' % (name, value))

molecule_type = DNA
topology = linear
data_file_division = INV
date = 12-NOV-2014
accessions = ['KM288867']
sequence_version = 1
keywords = ['']
source = Plasmodium falciparum (malaria parasite P. falciparum)
organism = Plasmodium falciparum
taxonomy = ['Eukaryota', 'Alveolata', 'Apicomplexa', 'Aconoidasida', 'Haemosporida', 'Plasmodiidae', 'Plasmodium', 'Plasmodium (Laverania)']
references = [Reference(title='Versatile control of Plasmodium falciparum gene expression with an inducible protein-RNA interaction', ...), Reference(title='Direct Submission', ...)]


8. Por fim, mas não menos importante, você pode acessar um pedaço fundamental de informação, a sequência:   

In [12]:
sequence = rec.seq

In [13]:
print(sequence)

ATATGTAAAACCAAAATAAATTAAACAGAATTTATTTTTAAAAGATTTATTTGTAACAATATTACCATGATGATTTATTAAAGTAAAATCACCACCTATTAATGGTTTTCCTATACTTTCCATTGTAGTTTTTCCAAAACCTTTTTTTTTATTTTTCTCATTTTGTAATAAATATAAATATAAATATAATGCTGGTATCGTTAATGATAAATTAATACCTAAACATTTCCATGTTAAAAATATATTTTTCTTTTTCGACTTTTTTTTATTATCATTTTCATCCTCAACTTTTTTACTCATACTACTATAATCATTTCTTATAAAACTTTTATAAAATATATTTTCCCTTTTTTTATAATTTCTACTAGTATCATAAACAATATTTCTTATATTCAATAATACATTAAAATTGTTATATCCATTTTTTTTTTTATTACATATTTTGTTTAAGCTATTTAAAAGCACCTTATTCATTTTTGAAATAATGAAACAGCAAAAAAACAAAAATAACGTAAACAATTATGATGTACATAAAATATATATATATATATATATATATATATATTATTTATATATTATTCACATTTACATTTATTTATTTATATCTTCCTATTTTATACGATAAAATAAATTCGATTTAAATAAAATATACGTTAAATTAAATTCTTATATCTTAACATTTCGACATACATTTATTAATTGATTTTATTTTATTTATTTTGTTATATCCATATAAATAAACGTATCGCACATGAGTTTTTTTTTTTTTTTTTTTTTTTTTTTATATAATACTTTTTAATAGTGAAACACATACATATTTTTTACCATTTTAAATATATATATGGTGTACCATTTTGGGTTAATATATTTTTTGTAATTATATATTAATATATACATAATAAATATAAATTAATATTATATATATATATATATATATATATATAAATAATTGGAGTGTCCTATTTTGAAGCTATGTCCTTTGGAATACAGCACTTTACAA